<a href="https://colab.research.google.com/github/fabiobasson/Bi-Master/blob/main/geological_image_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações 

In [90]:
import os
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from os import getcwd
import zipfile
import shutil
import numpy as np
import glob
import random
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
!rm -rf geological_similarity andesite gneiss/ marble/ quartzite/ rhyolite/ schist/

# **Coleta de dados do Kaglle**

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"fabiobasson","key":"d71cbfdfd23379909173204eac74cf96"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# **###  Download dos dados / descompactando os dados**

In [ ]:
#!kaggle datasets download tanyadayanand/geological-image-similarity
#! unzip -qq  geological-image-similarity

In [5]:
!kaggle datasets download fabiobasson/geologicalsimilarity
! unzip -qq  geologicalsimilarity

  0% 0.00/1.31M [00:00<?, ?B/s]
100% 1.31M/1.31M [00:00<00:00, 44.1MB/s]


# ** Criação da estrutura **

In [91]:
andesite_dir = glob.glob('andesite/*.jpg');gneiss_dir = glob.glob('gneiss/*.jpg')
marble_dir = glob.glob('marble/*.jpg');quartzite_dir = glob.glob('quartzite/*.jpg')
rhyolite_dir = glob.glob('rhyolite/*.jpg');
schist_dir = glob.glob('schist/*.jpg')
print(len(andesite_dir));print(len(gneiss_dir)); print(len(marble_dir)); print(len(quartzite_dir));  print(len(quartzite_dir)); print(len(schist_dir))


230
190
204
198
198
204


# **Nomes das Classes**
# andesite = 0  # gneiss =1 # marbler = 2
# quartzite = 3 # rhyolite = 4  # schist = 5 


In [97]:
class_names = [0,1,2,3,4,5]
class_names[4]

4

In [98]:
andesite_df=[];gneiss_df=[];marble_df=[];quartzite_df=[];rhyolite_df=[];schist_df=[]

for i in andesite_dir:
    andesite_df.append([i,class_names[0]])
for j in gneiss_dir:
   gneiss_df.append([j,class_names[1]])
for l in marble_dir:
    marble_df.append([l,class_names[2]])
for m in quartzite_dir:
    quartzite_df.append([m,class_names[3]])
for n in rhyolite_dir:
    rhyolite_df.append([n,class_names[4]])
for o in schist_dir:
    schist_df.append([o,class_names[5]])
   
df = andesite_df + gneiss_df + marble_df + quartzite_df + rhyolite_df + schist_df
random.shuffle(df)
df

[['rhyolite/0RO6A.jpg', 4],
 ['andesite/02Q0N.jpg', 0],
 ['gneiss/0RRWS.jpg', 1],
 ['schist/1OWQN.jpg', 5],
 ['quartzite/1RW3I.jpg', 3],
 ['quartzite/0PVU4.jpg', 3],
 ['schist/1YS7B.jpg', 5],
 ['quartzite/2BPDR.jpg', 3],
 ['rhyolite/1XOAT.jpg', 4],
 ['schist/1J8PC.jpg', 5],
 ['gneiss/0SZ87.jpg', 1],
 ['marble/1AZSB.jpg', 2],
 ['gneiss/0GV1I.jpg', 1],
 ['rhyolite/0EP7N.jpg', 4],
 ['andesite/1ZJ0G.jpg', 0],
 ['marble/1SYUI.jpg', 2],
 ['gneiss/0SA1U.jpg', 1],
 ['rhyolite/0M00X.jpg', 4],
 ['marble/0EHO3.jpg', 2],
 ['schist/1ZLQJ.jpg', 5],
 ['rhyolite/0SWNP.jpg', 4],
 ['marble/1O228.jpg', 2],
 ['gneiss/1RFQS.jpg', 1],
 ['andesite/1KGS3.jpg', 0],
 ['andesite/0Z2AO.jpg', 0],
 ['andesite/1I5G3.jpg', 0],
 ['quartzite/0H3MR.jpg', 3],
 ['gneiss/0ICYJ.jpg', 1],
 ['rhyolite/1QPDQ.jpg', 4],
 ['andesite/1W32X.jpg', 0],
 ['schist/0LXR7.jpg', 5],
 ['andesite/0TKR6.jpg', 0],
 ['gneiss/0NJLK.jpg', 1],
 ['rhyolite/1BQQM.jpg', 4],
 ['quartzite/1H503.jpg', 3],
 ['rhyolite/1SO67.jpg', 4],
 ['schist/1MJPC.jpg

In [94]:
data_df = pd.DataFrame(df,columns=['path','label'])
data_df

,path,label
0,andesite/1ZJVX.jpg,0
1,andesite/1CQSI.jpg,0
2,marble/0IGO9.jpg,2
3,gneiss/0QJ1R.jpg,1
4,marble/0SY6Y.jpg,2
...,...,...
1213,quartzite/1MRH6.jpg,3
1214,gneiss/0TV0H.jpg,1
1215,schist/0PCYI.jpg,5
1216,schist/2A7ZO.jpg,5


In [99]:
imagensrocha = data_df["path"].tolist()

In [100]:
def df_values(lista_nome_imagens):
    """Função para criar um dataframe com as dimensões das imagens de uma pasta.
    Parâmetros:
    folder_das_imagens(str): colocar a pasta onde as imagens estão salvas
    lista_nome_imagens(list): colocar a lista com o nome das imagens
    
    return
    
    df_dims(pd.DataFrame)
    
    """
    
    dic = {}
    dimensaoImagensLargura = []
    dimensaoImagensAltura = []
    nome = []
    pixels = [] 
    
    for imagem in lista_nome_imagens:
        enderecoDaImagem = imagem
        abrirImagem = Image.open(enderecoDaImagem)
        pixels= asarray(abrirImagem)
        
        nome.append(imagem)
        dimensaoImagensLargura.append(abrirImagem.size[0])
        dimensaoImagensAltura.append(abrirImagem.size[1])
        print(pixels)

    dic["Nome"] = nome
    dic["Largura"] = dimensaoImagensLargura
    dic["Altura"] = dimensaoImagensAltura
    dic["Pixels"] = pixels

    df_dims = pd.DataFrame(dic)
    
    return df_dims

In [101]:
# Utilizar a função criada para observar os valores das imagens
values = df_values(imagensrocha)
print(values)


A saída de streaming foi truncada nas últimas 5000 linhas.
  [ 68  50  28]
  [ 62  47  24]]]
[[[ 99  56  40]
  [ 86  54  31]
  [ 83  63  38]
  ...
  [114  88  65]
  [143 137 113]
  [115 114  94]]

 [[100  86  60]
  [ 92  66  43]
  [ 87  56  35]
  ...
  [102  86  60]
  [120 113  87]
  [106  94  72]]

 [[ 93 100  67]
  [ 91  69  45]
  [ 98  56  40]
  ...
  [109 102  76]
  [116 106  81]
  [122 101  80]]

 ...

 [[ 92  67  45]
  [ 99  77  54]
  [100  78  55]
  ...
  [125 105  80]
  [109  94  65]
  [116 110  78]]

 [[ 92  65  44]
  [ 87  71  46]
  [ 82  70  44]
  ...
  [128 127  97]
  [112 113  82]
  [ 97  96  66]]

 [[ 98  67  47]
  [103  89  63]
  [ 79  74  45]
  ...
  [119 120  89]
  [116 119  88]
  [100  96  69]]]
[[[ 81  63  43]
  [ 84  58  41]
  [ 79  47  32]
  ...
  [ 92  81  63]
  [108  97  79]
  [ 99  82  64]]

 [[ 93  75  55]
  [ 83  57  40]
  [ 78  46  31]
  ...
  [ 95  97  75]
  [ 94  87  68]
  [ 88  65  51]]

 [[ 78  60  40]
  [ 76  50  33]
  [ 80  51  35]
  ...
  [101 109  85]

Exception: ignored

#  Conhecendo os dados de imagem

In [ ]:
import PIL
from PIL import Image
# Carregar um imagem a partir do disco:
image = Image.open("gneiss/0QTQE.jpg")
print(image.size)
print(image.mode)
image

In [ ]:
from matplotlib import image
from matplotlib import pyplot
# Carregando imagem como um array NumPy:
data = image.imread("gneiss/0QTQE.jpg")
# Imprimindo o conteúdo do array NumPy:
print(data)

In [ ]:
# Imprimindo as propriedades do array de pixels:
print(data.dtype)
print(data.shape)
print(data.max())
print(data.min())

# Exibindo o array de pixels como uma imagem:
pyplot.imshow(data)

In [25]:
from numpy import asarray
image = Image.open("gneiss/0QTQE.jpg")
# Método asarray para realizar a conversão:
pixels= asarray(image)
print(pixels)
# Verificando a faixa de valores entre 0 e 255:
print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))


#Converte os valores inteiros em float para realizar a operação:
pixels = pixels.astype('float32')
# Normaliza a faixa de valores:
pixels /= pixels.max()
# Verificando a faixa de valores normalizada:
print('Data Type: %s' % pixels.dtype)
print('Min: %.3f, Max: %.3f' % (pixels.min(), pixels.max()))

[[[192 205 196]
  [184 197 188]
  [186 203 193]
  ...
  [184 210 207]
  [186 212 209]
  [185 211 208]]

 [[190 201 205]
  [188 203 206]
  [181 201 202]
  ...
  [195 206 202]
  [198 207 202]
  [197 206 201]]

 [[198 196 207]
  [194 201 209]
  [196 207 211]
  ...
  [196 181 178]
  [198 183 180]
  [196 181 176]]

 ...

 [[183 139 152]
  [177 149 163]
  [225 221 235]
  ...
  [184 177 185]
  [182 183 188]
  [174 179 183]]

 [[141  95 106]
  [134 102 115]
  [212 206 218]
  ...
  [170 154 165]
  [178 176 181]
  [171 177 177]]

 [[134  89  92]
  [126  89  97]
  [140 120 132]
  ...
  [170 160 171]
  [175 176 180]
  [164 175 171]]]
Data Type: uint8
Min: 32.000, Max: 251.000
Data Type: float32
Min: 0.127, Max: 1.000


# **Criando o Dataframe**

In [ ]:
data_df = pd.DataFrame(df,columns=['path','label'])
data_df

# **Limpeza de dados**
1 - Analisando os dados ausentes
2 - Descartando os dados ausentes
3 - Imputando dados
4 - Acrescentando colunas informativas

In [ ]:
import missingno as msno
ax = msno.matrix(data_df.sample(400))

In [ ]:
#! pip install pandas-profiling==2.8.0
#from pandas_profiling import ProfileReport
#ProfileReport (data_df)

# **Separação das amostras**

In [ ]:
X = data_df.drop(columns='label')
y = data_df.path

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=44)

# **Explorando os dados**

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#fig, ax = plt.subplots(figsize=(6,4))
#X.plot(kind="hist",ax=ax)

# **Data Argumentation**

## ** Esta função irá plotar as imagens num grid de 1 a 5 colunas **

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
IMG_SHAPE = 28
BATCH_SIZE = 20

In [ ]:
datagen = ImageDataGenerator(rescale=1./255,
                            featurewise_center =False ,
                            # samplewise_center =False ,
                            # featurewise_std_normalization =False ,
                            # samplewise_std_normalization =False ,
                            # zca_whitening =False ,
                            # zca_epsilon =1e -6 ,
                            # channel_shift_range =0. ,
                            # fill_mode = ’ nearest ’ ,
                            # cval =0. ,
                            rotation_range = 40,
                            width_shift_range = 0.2,
                            height_shift_range = 0.2,
                            #shear_range = 0.2,
                            #zoom_range = 0.2,
                            horizontal_flip = True,
                            vertical_flip =False , 
                            validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_dataframe(data_df,
                                              x_col='path',
                                              y_col='label',
                                              images_size=(28,28),
                                              #class_mode='binary',
                                              subset='training')

validation_generator = datagen.flow_from_dataframe(data_df,
                                                   x_col='path',
                                                   y_col='label',
                                                   images_size=(28,28),
                                                   #class_mode='binary',
                                                   subset='validation')

## Duvidas - Existe esse codigo abaixo

#test_generator = datagen.flow_from_dataframe(data_df,
                                             #x_col='path',
                                             #y_col='label',
                                             #images_size=(28,28)
                                             #class_mode='binary',
                                            #)

In [ ]:
augmented_images = [train_generator[0][0][0] for i in range(5)]
plotImages(augmented_images)

# **Imputação dos dados ???**

# **Normalização dos dados ???**

# **Pre Processamento dos dados**

# **Criar o modelo**

In [ ]:
inputs = tf.keras.Input(shape=(256, 256,3))
#x = augmented_images(inputs) # como conserto e passo isso
x = layers.experimental.preprocessing.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="sigmoid")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(256, (3,3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1)
])

# **Compilando o Modelo**

In [ ]:
model.compile(optimizer = Adam(learning_rate=0.0001), 
               loss = 'binary_crossentropy', 
               metrics =['accuracy'])

In [ ]:
model.summary()  

In [ ]:
callbacks_list = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.5,
                                         patience=4,
                                         verbose=0),
    tf.keras.callbacks.EarlyStopping(
        monitor="accuracy",
        patience=12,
        #restore_best_weights=True,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.h5',
        monitor="val_accuracy",
        save_best_only=True,
    ),

]

# **Avaliar o modelo**

In [ ]:
history = model.fit(train_generator,
                               epochs=5,
                               #verbose=0,
                               callbacks=callbacks_list, 
                               #validation_split=0.2,
                               #class_weight=class_weight_,
                               #workers=4,
                               use_multiprocessing=True,
                               validation_data=validation_generator)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# **Otimizar o modelo**

# **Avaliar o Modelo**

In [ ]:
model = tf.keras.models.load_model("best_model.h5")

val_acc = model.evaluate(validation_generator)[1]
print("validation_accuracy: " + str(val_acc))